In [124]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import re
from collections import Counter
from sklearn import metrics
from glob import glob as g

%matplotlib inline

### Create Individual CSVs

In [125]:
# Set patterns
timestamp_pattern = "[[](\d\d:\d\d)[]]"
user_pattern = "[[]\d\d:\d\d[]]\s[<](.*)[>]"
message_pattern = "[[]\d\d:\d\d[]]\s[<].*[>]\s(.*)"

In [230]:
# Get files

# Train
filepath = '/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/data/'
subfolders = ["train", "test", "dev"]

for subfolder in subfolders:
    filelist_ascii = g(filepath + subfolder + '/*ascii.txt')
    for filename in filelist_ascii:
        data = pd.read_csv(filename, header = None, names = ['raw'], delimiter="\t")
        data['timestamp'] = data['raw'].str.extract(timestamp_pattern, expand=True)
        data['user'] = data['raw'].str.extract(user_pattern, expand=True)
        data['message'] = data['raw'].str.extract(message_pattern, expand=True)
        data.to_csv(filename + ".csv")

### Aggregate CSVs

In [231]:
date_pattern = "(\d\d\d\d-\d\d-\d\d)"

# Train
filepath = '/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/data/train'
csvs_train = pd.DataFrame(g(filepath + '/*.csv'), columns=["path"])
csvs_train["date"] = csvs_train["path"].str.extract(date_pattern, expand=True)

# Test
filepath = '/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/data/test'
csvs_test = pd.DataFrame(g(filepath + '/*.csv'), columns=["path"])
csvs_test["date"] = csvs_test["path"].str.extract(date_pattern, expand=True)


# Dev
filepath = '/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/data/dev'
csvs_dev = pd.DataFrame(g(filepath + '/*.csv'), columns=["path"])
csvs_dev["date"] = csvs_dev["path"].str.extract(date_pattern, expand=True)


In [224]:
csvs_train.head(3)

,path,date
0,/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/data/train/2013-02-24.train-a.ascii.txt.csv,2013-02-24
1,/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/data/train/2013-07-10.train-a.ascii.txt.csv,2013-07-10
2,/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/data/train/2007-01-21.train-c.ascii.txt.csv,2007-01-21


In [227]:
#Aggregate files
rootfile = '/Users/m/Desktop/MSCA Capstone/Chat-Disentangle-main/'

agg_train = pd.DataFrame()
for i in range(len(csvs_train.path)):
    data = pd.read_csv(csvs_train.path[i], header = 0)
    data['date'] = pd.Series([csvs_train.date[i] for x in range(len(data))])
    agg_train = agg_train.append(data)
agg_train.to_csv(rootfile + "agg_train.csv")
    
agg_test = pd.DataFrame()
for i in range(len(csvs_test.path)):
    data = pd.read_csv(csvs_test.path[i], header = 0)
    data['date'] = pd.Series([csvs_test.date[i] for x in range(len(data))])
    agg_test = agg_test.append(data)
agg_test.to_csv(rootfile + "agg_test.csv")
    
agg_dev = pd.DataFrame()
for i in range(len(csvs_dev.path)):
    data = pd.read_csv(csvs_dev.path[i], header = 0)
    data['date'] = pd.Series([csvs_dev.date[i] for x in range(len(data))])
    agg_dev = agg_dev.append(data)
agg_dev.to_csv(rootfile + "agg_dev.csv")

In [228]:
agg_dev

,Unnamed: 0,raw,timestamp,user,message,date
0,0,[06:22] <cactusfrog> i am that fucking stupid,06:22,cactusfrog,i am that fucking stupid,2009-03-03
1,1,[06:22] <cactusfrog> its unfair,06:22,cactusfrog,its unfair,2009-03-03
2,2,[06:22] <cactusfrog> that people are smarter then me,06:22,cactusfrog,that people are smarter then me,2009-03-03
3,3,"[06:23] <Titan8990> Flannel, sorry, to me vanilla = downloaded from kernel.org",06:23,Titan8990,"Flannel, sorry, to me vanilla = downloaded from kernel.org",2009-03-03
4,4,"[06:23] <Flannel> Titan8990: right, but when he said ""Yes, out of the box"" he obviously understands vanilla as default U kernel",06:23,Flannel,"Titan8990: right, but when he said ""Yes, out of the box"" he obviously understands vanilla as default U kernel",2009-03-03
...,...,...,...,...,...,...
1245,1245,[04:50] <HrdwrBoB> good to hear :),04:50,HrdwrBoB,good to hear :),2004-11-15
1246,1246,[04:51] <HrdwrBoB> janice: if you start synaptic,04:51,HrdwrBoB,janice: if you start synaptic,2004-11-15
1247,1247,[04:51] <HrdwrBoB> and you enable 'restricted',04:51,HrdwrBoB,and you enable 'restricted',2004-11-15
1248,1248,[04:51] <HrdwrBoB> you can install ndiswrapper,04:51,HrdwrBoB,you can install ndiswrapper,2004-11-15


In [ ]:
# Cleanup files